In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!git clone https://github.com//Tutorial-Book-Utils

In [ ]:
!pip install gdown    

In [ ]:
#!python Tutorial-Book-Utils/PL_data_loader.py --data FaceMaskDetection

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
#!unzip -q Face\ Mask\ Detection.zip

In [ ]:
! cp -R /kaggle/input/tbx-11/* /kaggle/working

In [ ]:
%cd /kaggle/input/bboxaggregated13aug

In [ ]:
%cd /kaggle/input/bboxdf-final-prebox
mytraindf = pd.read_csv("bbox_df_train_finalbackupprocessed.csv")
mytraindf.head(5)

In [ ]:
mytraindf_new = mytraindf[mytraindf['file_name'].str.contains('tb')] 
mytraindf_new

#df = df[df['Credit-Rating'].str.contains('Fair')]

In [ ]:
np.unique(mytraindf_new.newclass_name)

In [ ]:
np.unique(mytraindf.newclass_name)

### 2.3  Bounding Box Diagram 

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from bs4 import BeautifulSoup

img_list = sorted(glob.glob('TBX11K/imgs/tb/*'))
annot_list = sorted(glob.glob('TBX11K/annotations/xml/*'))

#img_list = sorted(glob.glob('TBX11K/imgs/tb/*'))
#annot_list = sorted(glob.glob('TBX11K/annotations/json*'))

In [ ]:
print(len(img_list))
print(len(annot_list))

In [ ]:
print(img_list[:10])
print(annot_list[:10])

## Visualize Bounding boxes

In [ ]:
def generate_box(obj, width, height):
    width_ratio = 512 / float(width)
    height_ratio = 512 / float(height)
    
    xmin = float(obj.find('xmin').text)
    ymin = float(obj.find('ymin').text)
    xmax = float(obj.find('xmax').text)
    ymax = float(obj.find('ymax').text)
    
    true_xmin = xmin * width_ratio
    true_ymin = ymin * height_ratio
    true_xmax = true_xmin + ((xmax - xmin) * width_ratio)
    true_ymax = true_ymin + ((ymax - ymin) * height_ratio)

    return [true_xmin, true_ymin, true_xmax, true_ymax]

def generate_label(obj):
    if obj.find('name').text == "ActiveTuberculosis":
        return 1
    elif obj.find('name').text == "ObsoletePulmonaryTuberculosis":
        return 2
    return 0

def generate_target(file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, "html.parser")
        width_element = soup.find('width')
        if width_element:
            width = width_element.get_text()
            print(f"width text: {width}")
        else:
            width = 1811
            print("width not found")

        height_element = soup.find('height')
        if height_element:
            height = height_element.get_text()
            print(f"height text: {height}")
        else:
            height = 2022
            print("height not found")
    
        objects = soup.find_all("object")
        num_objs = len(objects)

        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i, width, height))
            labels.append(generate_label(i))

        print("Boxes :" , boxes)
        print("Labels : ", labels)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        print("target : ", target)
        return target

def plot_image(img_path, annotation):
    
    img = mpimg.imread(img_path)
    print(img.shape)
    fig,ax = plt.subplots(1)
    ax.imshow(img)

    
    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 0 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        
        elif annotation['labels'][idx] == 1 :            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        else :        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')

        ax.add_patch(rect)

    plt.show()

In [ ]:
imgID = img_list.index('TBX11K/imgs/tb/tb0096.png')
bbox = generate_target(annot_list[imgID])
plot_image(img_list[imgID], bbox)

###  3 - Albumentations 

In [ ]:
!pip install --upgrade albumentations

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from bs4 import BeautifulSoup

def generate_box(obj, width, height):
    width_ratio = 512 / float(width)
    height_ratio = 512 / float(height)
    
    xmin = float(obj.find('xmin').text)
    ymin = float(obj.find('ymin').text)
    xmax = float(obj.find('xmax').text)
    ymax = float(obj.find('ymax').text)
    
    true_xmin = xmin * width_ratio
    true_ymin = ymin * height_ratio
    true_xmax = true_xmin + ((xmax - xmin) * width_ratio)
    true_ymax = true_ymin + ((ymax - ymin) * height_ratio)

    return [true_xmin, true_ymin, true_xmax, true_ymax]


def generate_label(obj):
    if obj.find('name').text == "ActiveTuberculosis":
        return 1
    elif obj.find('name').text == "ObsoletePulmonaryTuberculosis":
        return 2
    return 0

def generate_target(file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, "html.parser")
        width_element = soup.find('width')
        if width_element:
            width = width_element.get_text()
        else:
            width = 1811

        height_element = soup.find('height')
        if height_element:
            height = height_element.get_text()
        else:
            height = 2022
            
        objects = soup.find_all("object")
        num_objs = len(objects)

        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i, width, height))
            labels.append(generate_label(i))

        boxes = torch.as_tensor(boxes, dtype=torch.float32) 
        labels = torch.as_tensor(labels, dtype=torch.int64) 
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        
        #print("target : ", target)
        
        return target

def plot_image_from_output(img, annotation):
    
    img = img.permute(1,2,0)
    
    fig,ax = plt.subplots(1)
    ax.imshow(img)
    
    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 0 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        
        elif annotation['labels'][idx] == 1 :
            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        else :
        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')

        ax.add_patch(rect)

    plt.show()

In [ ]:
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
import albumentations
import albumentations.pytorch
from matplotlib import pyplot as plt
import os
import random

class TorchvisionMaskDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))
        self.transform = transform
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)
        
        if 'test' in self.path:
            label_path = os.path.join("test_annotations/", file_label)
        else:
            label_path = os.path.join("/kaggle/working/TBX11K/annotations/xml/", file_label)

        img = Image.open(img_path).convert("RGB")
        
        target = generate_target(label_path)
        start_t = time.time()
        if self.transform:
            img = self.transform(img)
        total_time = (time.time() - start_t)

        return img, target, total_time

In [ ]:
torchvision_transform = transforms.Compose([
    transforms.Resize((512, 512)), 
    #transforms.RandomCrop(224),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomHorizontalFlip(p = 1),
    transforms.ToTensor(),
])

torchvision_dataset = TorchvisionMaskDataset(
    path = '/kaggle/working/TBX11K/imgs/tb/',
    transform = torchvision_transform
)

In [ ]:
only_totensor = transforms.Compose([transforms.ToTensor()])

torchvision_dataset_no_transform = TorchvisionMaskDataset(
    path = '/kaggle/working/TBX11K/imgs/tb/',
    transform = only_totensor
)

img, annot, transform_time = torchvision_dataset_no_transform[0]
print('Before applying transforms')
plot_image_from_output(img, annot)

In [ ]:
img, annot, transform_time = torchvision_dataset[0]

print('After applying transforms')
plot_image_from_output(img, annot)

In [ ]:
total_time = 0
for i in range(100):
  sample, _, transform_time = torchvision_dataset[0]
  total_time += transform_time

print("torchvision time: {} ms".format(total_time*10))

###### 3.1.2 Albumentations

In [ ]:
class AlbumentationsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))
        self.transform = transform
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)
        print(img_path)

        if 'test' in self.path:
            label_path = os.path.join("test_annotations/", file_label)
        else:
            label_path = os.path.join("/kaggle/working/TBX11K/annotations/xml/", file_label)
        
        
        # Read an image with OpenCV
        image = cv2.imread(img_path)
        if image is None or image.size == 0:
            print("Image is empty or could not be loaded.")
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            target = generate_target(label_path)

            start_t = time.time()
            if self.transform:
                augmented = self.transform(image=image)
                total_time = (time.time() - start_t)
                image = augmented['image']
                #print("Inside AlbumentationsDataset Image after Transform: ", image)   

        return image, target, total_time

In [ ]:
# Same transform with torchvision_transform
albumentations_transform = albumentations.Compose([
    albumentations.Resize(300, 300), 
    #albumentations.RandomCrop(224, 224),
    albumentations.ColorJitter(p=1), 
    albumentations.HorizontalFlip(p=1), 
    albumentations.pytorch.transforms.ToTensorV2()
])

In [ ]:
# Before applying transforms
img, annot, transform_time = torchvision_dataset_no_transform[0]
print(img)
plot_image_from_output(img, annot)

In [ ]:
# After applying transforms
albumentation_dataset = AlbumentationsDataset(
    path = '/kaggle/working/TBX11K/imgs/tb/',
    transform = albumentations_transform
)

img, annot, transform_time = albumentation_dataset[0]
plot_image_from_output(img, annot)

In [ ]:
total_time = 0
for i in range(5):
    sample, _, transform_time = albumentation_dataset[0]
    total_time += transform_time

print("albumentations time/sample: {} ms".format(total_time*10))

### 3.1.3 Probability based Augmentations 

In [ ]:
albumentations_transform_oneof = albumentations.Compose([
    albumentations.Resize(300, 300), 
    #albumentations.RandomCrop(224, 224),
    albumentations.OneOf([
                          albumentations.HorizontalFlip(p=1),
                          albumentations.RandomRotate90(p=1),
                          albumentations.VerticalFlip(p=1)            
    ], p=1),
    albumentations.OneOf([
                          albumentations.MotionBlur(p=1),
                          albumentations.OpticalDistortion(p=1),
                          albumentations.GaussNoise(p=1)                 
    ], p=1),
    albumentations.pytorch.ToTensorV2()
])

In [ ]:
albumentation_dataset_oneof = AlbumentationsDataset(
    path = '/kaggle/working/TBX11K/imgs/tb/',
    transform = albumentations_transform_oneof
)

num_samples = 10
fig, ax = plt.subplots(1, num_samples, figsize=(25, 5))
for i in range(num_samples):
  ax[i].imshow(transforms.ToPILImage()(albumentation_dataset_oneof[0][0]))
  ax[i].axis('off')


## 3.2. Bounding Box Augmentation

In [ ]:
import cv2

In [ ]:
class BboxAugmentationDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))
        self.transform = transform
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)

        if 'test' in self.path:
            label_path = os.path.join("test_annotations/", file_label)
        else:
            label_path = os.path.join("/kaggle/working/TBX11K/annotations/xml/", file_label)
        
        image = cv2.imread(img_path)
        
        if image is None:
            raise ValueError(f"Image not found or could not be loaded: {img_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        target = generate_target(label_path)

        if self.transform:
            transformed = self.transform(image = image, bboxes = target['boxes'], labels = target['labels'])
            image = transformed['image']
            target = {'boxes':transformed['bboxes'], 'labels':transformed['labels']}
        
            
        return image, target

In [ ]:
bbox_transform = albumentations.Compose(
    [albumentations.HorizontalFlip(p=1),
     albumentations.Rotate(p=1),
     albumentations.pytorch.transforms.ToTensorV2()],
    bbox_params=albumentations.BboxParams(format='pascal_voc', label_fields=['labels']),
)

In [ ]:
bbox_transform_dataset = BboxAugmentationDataset(
    path = '/kaggle/working/TBX11K/imgs/tb/',
    transform = bbox_transform
)

img, annot = bbox_transform_dataset[0]
plot_image_from_output(img, annot)

## 3.3 Data Seperation

In [ ]:
print(len(os.listdir('/kaggle/working/TBX11K/annotations/xml/')))
print(len(os.listdir('/kaggle/working/TBX11K/imgs/tb/')))

In [ ]:
%cd /kaggle/working/

!mkdir test_images
!mkdir test_annotations

In [ ]:
import random
random.seed(1234)
idx = random.sample(range(800), 300)
print(len(idx))
print(idx[:10])

In [ ]:
len(np.array(sorted(os.listdir('/kaggle/working/TBX11K/imgs/tb/'))))

In [ ]:
!pwd

In [ ]:
len(np.array(sorted(os.listdir('/kaggle/working/TBX11K/imgs/tb/'))))

In [ ]:
import numpy as np
import shutil

for img in np.array(sorted(os.listdir('/kaggle/working/TBX11K/imgs/tb/')))[idx]:
    shutil.move('/kaggle/working/TBX11K/imgs/tb/'+ img, '/kaggle/working/test_images/'+img)

for annot in np.array(sorted(os.listdir('/kaggle/working/TBX11K/annotations/xml/')))[idx]:
    shutil.move('/kaggle/working/TBX11K/annotations/xml/'+ annot, '/kaggle/working/test_annotations/'+ annot)

In [ ]:
print(len(os.listdir('/kaggle/working/TBX11K/annotations/xml/')))
print(len(os.listdir('/kaggle/working/TBX11K/imgs/tb/')))
print(len(os.listdir('test_annotations')))
print(len(os.listdir('test_images')))

In [ ]:
print(len(os.listdir('/kaggle/working/TBX11K/imgs/tb/')))

In [ ]:
from tqdm import tqdm
import pandas as pd
from collections import Counter

def get_num_objects_for_each_class(dataset):

    total_labels = []

    for img, annot in tqdm(dataset, position = 0, leave = True):
        total_labels += [int(i) for i in annot['labels']]

    return Counter(total_labels)


train_data =  BboxAugmentationDataset(
    path = '/kaggle/working/TBX11K/imgs/tb/'
)

test_data =  BboxAugmentationDataset(
    path = '/kaggle/working/test_images/'
)

train_objects = get_num_objects_for_each_class(train_data)
test_objects = get_num_objects_for_each_class(test_data)

print('\n Object in training data', train_objects)
print('\n Object in test data', test_objects)

In [ ]:
#np.array(sorted(os.listdir('images')))

In [ ]:
import os
import random
import numpy as np
import shutil

print(len(os.listdir('/kaggle/working/TBX11K/annotations')))
print(len(os.listdir('/kaggle/working/TBX11K/imgs')))

#!mkdir test_images
#!mkdir test_annotations


random.seed(1234)
idx = random.sample(range(853), 170)

#for img in np.array(sorted(os.listdir('images')))[idx]:
    #shutil.move('images/'+img, 'test_images/'+img)

#for annot in np.array(sorted(os.listdir('/kaggle/working/test_annotations')))[idx]:
    #shutil.move('/kaggle/working/TBX11K/annotations/'+annot, '/kaggle/working/test_annotations/'+annot)

print(len(os.listdir('/kaggle/working/TBX11K/annotations')))
print(len(os.listdir('/kaggle/working/TBX11K/imgs')))
print(len(os.listdir('/kaggle/working/test_annotations')))
print(len(os.listdir('/kaggle/working/test_images')))

In [ ]:
print(len(os.listdir('/kaggle/working/TBX11K/annotations')))
print(len(os.listdir('/kaggle/working/TBX11K/imgs')))
print(len(os.listdir('/kaggle/working/test_annotations')))
print(len(os.listdir('/kaggle/working/test_images')))

In [ ]:
print(len(os.listdir('/kaggle/working/TBX11K/annotations')))
print(len(os.listdir('/kaggle/working/TBX11K/imgs')))
print(len(os.listdir('/kaggle/working/test_annotations')))
print(len(os.listdir('/kaggle/working/test_images')))

## 4.3 Defining Dataset Class

In [ ]:
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
import albumentations
import albumentations.pytorch
from matplotlib import pyplot as plt
import os
import random

In [ ]:
# Same transform with torchvision_transform
albumentations_transform_TRAIN = albumentations.Compose([
    albumentations.Resize(512, 512), 
    albumentations.RandomCrop(500, 500),###
    albumentations.ColorJitter(p=1.0), 
    albumentations.HorizontalFlip(p=1), ##
    albumentations.RandomBrightnessContrast(p=0.2),     
    albumentations.RandomFog(p = 1),         ##
  #A.VerticalFlip(p = 1),
  #albumentations.RandomBrightness(p = 1),
  #albumentations.RandomContrast(limit = 0.6, p = 1),
  #albumentations.RandomCrop(p = 1,height = 450, width = 450),
  #albumentations.Rotate(p = 1, limit = 90),
    albumentations.RGBShift(p = 1.0),
    albumentations.HueSaturationValue(p=0.35),
    albumentations.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
    albumentations.ToGray(p = 1),
    albumentations.pytorch.transforms.ToTensorV2()],    
    bbox_params=albumentations.BboxParams(format='pascal_voc', label_fields=['labels'])
    )


# Same transform with torchvision_transform
albumentations_transform_TEST = albumentations.Compose([
    albumentations.Resize(512, 512),   
    albumentations.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
    albumentations.pytorch.transforms.ToTensorV2()],
    bbox_params=albumentations.BboxParams(format='pascal_voc', label_fields=['labels'])
    )


#bbox_transform = albumentations.Compose(
    #[albumentations.HorizontalFlip(p=1),
    #albumentations.Rotate(p=1),
     #albumentations.pytorch.transforms.ToTensorV2()],
   # bbox_params=albumentations.BboxParams(format='pascal_voc', label_fields=['labels']),
#)

In [ ]:
Image Value Distribution¶



In [ ]:
mean_val = []
std_dev_val = []
max_val = []
min_val = []

for i in range(0, samples):
    mean_val.append(data['image'][i].mean())
    std_dev_val.append(np.std(data['image'][i]))
    max_val.append(data['image'][i].max())
    min_val.append(data['image'][i].min())

imageEDA = data.loc[:,['image','corona_result','path']]
imageEDA['mean'] = mean_val
imageEDA['stedev'] = std_dev_val
imageEDA['max'] = max_val
imageEDA['min'] = min_val

imageEDA['subt_mean'] = imageEDA['mean'].mean() - imageEDA['mean']

In [ ]:
##from : https://www.kaggle.com/code/danushkumarv/covid-19-cnn-grad-cam-viz#5-%7C-Conclusion

##and from https://www.kaggle.com/code/sana306/detection-of-covid-positive-cases-using-dl#Saving-Model

import albumentations as A
chosen_image = cv2.imread("/kaggle/input/tbx-11/TBX11K/imgs/tb/tb0005.png")

albumentation_list = [A.RandomFog(p = 1),
                      A.VerticalFlip(p = 1),
                      A.RandomBrightness(p = 1),
                      A.RandomContrast(limit = 0.6, p = 1),
                      A.RandomCrop(p = 1,height = 450, width = 450),
                      A.Rotate(p = 1, limit = 90),
                      A.RGBShift(p = 1),
                      A.HueSaturationValue(p=0.35),
                      A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
                      A.ToGray(p = 1)]

img_matrix_list = []
bboxes_list = []
for aug_type in albumentation_list:
    img = aug_type(image = chosen_image)['image']
    img_matrix_list.append(img)

img_matrix_list.insert(0,chosen_image)    

titles_list = ["Original", "VerticalFlip", "RandomFog",
               "RandomContrast", "RandomBrightness", 
               "RandomCrop", "Rotate", "RGBShift","HueSaturationValue","Gray"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 5, main_title = "Different Types of Augmentations")

In [ ]:
def plot_multiple_img(img_matrix_list, title_list, ncols, main_title = ""):
    
    fig, myaxes = plt.subplots(figsize = (15, 8), nrows = 2, ncols = ncols, squeeze = False)
    fig.suptitle(main_title, fontsize = 18)
    fig.subplots_adjust(wspace = 0.3)
    fig.subplots_adjust(hspace = 0.3)
    
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize = 15)
        
    plt.show()

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from bs4 import BeautifulSoup
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
from matplotlib import pyplot as plt
import os

def generate_box(obj, width, height):
    width_ratio = 512 / float(width)
    height_ratio = 512 / float(height)
    
    xmin = float(obj.find('xmin').text)
    ymin = float(obj.find('ymin').text)
    xmax = float(obj.find('xmax').text)
    ymax = float(obj.find('ymax').text)
    
    true_xmin = xmin * width_ratio
    true_ymin = ymin * height_ratio
    true_xmax = true_xmin + ((xmax - xmin) * width_ratio)
    true_ymax = true_ymin + ((ymax - ymin) * height_ratio)

    return [true_xmin, true_ymin, true_xmax, true_ymax]

def generate_label(obj):
    if obj.find('name').text == "ActiveTuberculosis":
        return 0
    elif obj.find('name').text == "ObsoletePulmonaryTuberculosis":
        return 1
    return 2

def generate_target(file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, "html.parser")
        
        width_element = soup.find('width')
        if width_element:
            width = width_element.get_text()
        else:
            print(" i am here")
            width = 1811

        height_element = soup.find('height')
        if height_element:
            height = height_element.get_text()
        else:
            print(" i am here")
            height = 2022
            
        objects = soup.find_all("object")

        num_objs = len(objects)
        #print("File name in generate_target : ",file )
        boxes = []
        labels = []
        for i in objects:            
            boxes.append(generate_box(i, width, height))
            labels.append(generate_label(i))
              
        boxes = torch.as_tensor(boxes, dtype=torch.float32) 
        labels = torch.as_tensor(labels, dtype=torch.int64) 
    
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
                
        
        return target

def plot_image_from_output(img, annotation):
    
    img = img.cpu().permute(1,2,0)
    
    rects = []

    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 0 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        
        elif annotation['labels'][idx] == 1 :
            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        else :
        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')

        rects.append(rect)

    return img, rects

class MaskDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))
        self.transform = transform
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)
        
        if 'test' in self.path:
            label_path = os.path.join("/kaggle/working/test_annotations/", file_label)
        else:
            label_path = os.path.join("/kaggle/working/TBX11K/annotations/xml/", file_label)
            #print("Label path : ",label_path)

        img = Image.open(img_path).convert("RGB")
        
        #print("Image in Mask: ",img )
        #print("Img path in Mask: ",img_path )
        #print("File_Label in MaskDataset : " , label_path)
        
        #print("Image in MaskDataset : ", img_path)
        target = generate_target(label_path)
        #print("Image in Maskdataset :", img_path )
        #print("Target in Maskdataset:", target )
        boxes = np.array(target['boxes'])
        labels = np.array(target['labels'])              
        #print("Target in MaskDataset : ", target)
        to_tensor = torchvision.transforms.ToTensor()

        if self.transform:
            #print("IN TRANSFORM :" ,self.transform)
            #img, transform_target = self.transform(image = np.array(img), boxes = np.array(target['boxes']))
            augmented = self.transform(image = np.array(img), bboxes = boxes , labels = labels)
            #print("hi.. this is rachana")
            img_augmented = augmented['image']
            box_augmented = augmented['bboxes']
            lbl_augmented = augmented['labels']  
            img = img_augmented
            #if (img_path == '/kaggle/working/TBX11K/imgs/tb/tb0206.png'):
                         
            #target = {'boxes':augmented['bboxes'], 'labels':augmented['labels']}
            target['boxes'] = torch.as_tensor(box_augmented, dtype=torch.float32)   # 16/8                            
            target['labels'] = torch.as_tensor(lbl_augmented, dtype=torch.int64) # 16/8
            
            #print(" Inside Transform :",img )
            #print(" Inside Transform :",target['boxes'] )
            #print(" Inside Transform :",target['labels'] )
            
            #target = {'boxes':torch.as_tensor(augmented['bboxes'],dtype=torch.float32), 'labels': torch.as_tensor(augmented['labels'],dtype=torch.int64)}
           
            
        # change to tensor
        #img = to_tensor(img) 
        #print("Image shape : ", img.shape)
        
        return img, target

def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset('/kaggle/working/TBX11K/imgs/tb/', transform=albumentations_transform_TRAIN)
test_dataset =MaskDataset('/kaggle/working/test_images/',transform=albumentations_transform_TEST)


data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, collate_fn=collate_fn)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn)

In [ ]:
len(dataset)

In [ ]:
len(data_loader)

### 4.4 Import Model

In [ ]:
torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0

In [ ]:
import torch
import torchvision

import torchaudio

In [ ]:
torchvision.__version__

In [ ]:
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
retina = torchvision.models.detection.retinanet_resnet50_fpn(num_classes = 2, pretrained=False, pretrained_backbone = True)

## 4.5 Transfer Learning

In [ ]:
"""
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_epochs = 2
retina.to(device)
    
# parameters
params = [p for p in retina.parameters() if p.requires_grad] # select parameters that require gradient calculation
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)
to_tensor = torchvision.transforms.ToTensor()

# about 4 min per epoch on Colab GPU
for epoch in range(num_epochs):
    start = time.time()
    retina.train()

    i = 0    
    epoch_loss = 0
    for images, targets, img_path in data_loader:
        #print("Images in DL : ",img_path)
             
        images = list(image.to(device) for image in images)
        
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        #print("Images : ", images)
        #print("Targets : ", targets)
        loss_dict = retina(images, targets) 

        losses = sum(loss for loss in loss_dict.values()) 

        i += 1

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        epoch_loss += losses 
    print(epoch_loss, f'time: {time.time() - start}')

In [ ]:
%cd working

In [ ]:
import torch
import time
#from torchvision.models.detection.retinanet import RetinaNetLoss

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_epochs = 5
retina.to(device)
    
# parameters
params = [p for p in retina.parameters() if p.requires_grad] # select parameters that require gradient calculation
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

# Loss function
#criterion = RetinaNetLoss(num_classes=3)

len_dataloader = len(data_loader)

# about 4 min per epoch on Colab GPU
for epoch in range(num_epochs):
    start = time.time()
    retina.train()

    i = 0    
    epoch_loss = 0
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]        
        #print("In Training loop :", images)
        #print("In Training loop :", targets)
        empty_found = False
        for t in targets:
            #print("All target items : ", t.items)
            for k, v in t.items():
                #k = k.to(device)
                #print("K value :", k)
                #print("V value :", v)
                if v.size(0) == 0:
                    #print(f"Empty tensor found for the dictionaries.")
                    empty_found = True

        if empty_found:
            print(" empty tensors found in the 'targets' list of dictionaries.")
        else:           
            
            
            # Forward pass for this image and its targets
            loss_dict = retina(images, targets)
            
            #loss_dict = retina(images_batch, targets_batch) 
            
            #print("Loss Dictionary :", loss_dict)

            losses = sum(loss for loss in loss_dict.values()) 

            #print("Losses :", losses)
            i += 1

            optimizer.zero_grad()
            losses.backward()
            
            ## Print gradients of model parameters
            #for name, param in retina.named_parameters():
                #if param.grad is not None:
                    #print(f"Gradient for parameter '{name}':")
                    #print(param.grad)

                    ## Check for NaN or infinite gradients
                    #if torch.isnan(param.grad).any() or torch.isinf(param.grad).any():
                        #print(f"NaN or Infinite gradients detected for parameter '{name}'!")
                #else:
                    #print(f"No gradient for parameter '{name}'")
            
            
            
            torch.nn.utils.clip_grad_norm_(retina.parameters(), max_norm=1.0)  # Clip gradients
            optimizer.step()

            epoch_loss += losses 
    print("Epoch : " , epoch)
    print(epoch_loss, f'time: {time.time() - start}')

In [ ]:
#torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients

In [ ]:
!pwd

In [ ]:
%cd /kaggle/working

In [ ]:
torch.save(retina.state_dict(),f'retina_{num_epochs}.pt')

In [ ]:
#%cd /kaggle/working/TBX11K/annotations/xml/
#!ls

In [ ]:
torch.save(retina.state_dict(),f'retina_{num_epochs}.pt')
retina.load_state_dict(torch.load(f'retina_{num_epochs}.pt'))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
retina.to(device)

In [ ]:
torch.save(retina.state_dict(),f'retina_{num_epochs}_backup_newalbu.pt')

### 4.6 Inference

In [ ]:
%cd ..

In [ ]:
#%cd ..
!pwd
!ls

In [ ]:
def make_prediction(model, img, threshold):
    model.eval()
    #print("Image for prediction : ",img )
    preds = model(img)
    #print("it is inside prediction")
    #print("Prediction : ", preds)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : #select idx which meets the threshold
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]


    return preds

In [ ]:
preds_adj

In [ ]:
!ls

In [ ]:
retina.load_state_dict(torch.load(f'retina_{num_epochs}.pt'))

from tqdm import tqdm

labels = []
preds_adj_all = []
annot_all = []

for im, annot in tqdm(test_data_loader, position = 0, leave = True):
    im = list(img.to(device) for img in im)
    #print("Batch Index:", batch_idx)
    #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]
    #print("Test Image :" , im)
    #print(" Test Loader : ",annot )
    
    # Move annotations to CPU
    annot = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in annot]
    
    for t in annot:
        labels += t['labels']

    with torch.no_grad():
        preds_adj = make_prediction(retina, im, 0.4)
        #print("Test Image Prediction :" , preds_adj)
        #devType = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
        
                
        preds_adj_all.append(preds_adj)
        annot_all.append(annot)
        #print(" Test Loader annot all: ",annot_all )
        #print(" Test Loader : ",len(annot) )
        print(preds_adj)

In [ ]:
preds_adj_all

In [ ]:
nrows = 8
ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))

batch_i = 0
for im, annot in test_data_loader:
    pos = batch_i * 4 + 1
    for sample_i in range(len(im)) :
        print(im[sample_i])
        print(annot[sample_i])
        img, rects = plot_image_from_output(im[sample_i], annot[sample_i])
        axes[(pos)//2, 1-((pos)%2)].imshow(img)
        for rect in rects:
            axes[(pos)//2, 1-((pos)%2)].add_patch(rect)
        
        img, rects = plot_image_from_output(im[sample_i], preds_adj_all[batch_i][sample_i])
        axes[(pos)//2, 1-((pos+1)%2)].imshow(img)
        for rect in rects:
            axes[(pos)//2, 1-((pos+1)%2)].add_patch(rect)

        pos += 2

    batch_i += 1
    if batch_i == 4:
        break

# remove xtick, ytick
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

colnames = ['True', 'Pred']

for idx, ax in enumerate(axes[0]):
    ax.set_title(colnames[idx])

plt.tight_layout()
plt.show()

In [ ]:
#preds_adj_all

In [ ]:
device = torch.device('cpu') if torch.cuda.is_available() else torch.device('cpu')
#retina.to(device)
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
## DO NOT CHANGE - CHAT

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_image_from_output_test(img, boxes, scores, threshold=0.5):
    img = img.cpu().permute(1, 2, 0).numpy()
    rects = []

    for i in range(boxes.size(0)):
        xmin, ymin, xmax, ymax = boxes[i]
        score = scores[i].item()  # Convert the score tensor to a scalar
        if score > threshold:  # Adjust the threshold as needed
            rect = patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none'
            )
            rects.append(rect)

    return img, rects


In [ ]:
#preds_adj_all

In [ ]:
## DO NOT CHANGE - CHAT

# WORKING ...ALTERNATE ROW
nrows = 8
ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))

batch_i = 0
for im, annot in test_data_loader:
    pos = batch_i * 4 + 1
    #pos = batch_i * (nrows // 2) * ncols + sample_i * ncols + 1  # Calculate the correct position
    im_cpu = tuple(t.cpu() for t in im)
    boxes_cpu = annot[0]['boxes'].cpu()
    labels_cpu = annot[0]['labels'].cpu()
    
    preds_adj_all_cpu = []
    for preds_adj_dict in preds_adj_all[batch_i]:
        preds_adj_cpu = {
            'boxes': preds_adj_dict['boxes'].cpu(),
            'scores': preds_adj_dict['scores'].cpu()
        }
        preds_adj_all_cpu.append(preds_adj_cpu)

    for sample_i in range(len(im_cpu)):
        img, rects = plot_image_from_output_test(im_cpu[sample_i], annot[sample_i]['boxes'],annot[sample_i]['labels'],) #preds_adj_all_cpu[sample_i]['boxes'], preds_adj_all_cpu[sample_i]['scores'])
        print(im_cpu ,annot[sample_i]['boxes'], annot[sample_i]['labels'] )
        axes[(pos)//2, 1-((pos)%2)].imshow(img)
        
        for rect in rects:
            axes[(pos)//2, 1-((pos)%2)].add_patch(rect)
        
        img, rects = plot_image_from_output_test(im_cpu[sample_i], preds_adj_all_cpu[sample_i]['boxes'], preds_adj_all_cpu[sample_i]['scores'])
        axes[(pos)//2, 1-((pos+1)%2)].imshow(img)
        
        for rect in rects:
            axes[(pos)//2, 1-((pos+1)%2)].add_patch(rect)

        pos += 1

    batch_i += 1
    if batch_i == 4:
        break

# remove xtick, ytick
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

colnames = ['True', 'Pred']

for idx, ax in enumerate(axes[0]):
    ax.set_title(colnames[idx])

plt.tight_layout()
plt.show()


In [ ]:
def plot_image_from_output_test(img, boxes, scores, threshold=0.5):
    img = img.cpu().permute(1, 2, 0).numpy()
    rects = []
    
    for i in range(boxes.size(0)):
        xmin, ymin, xmax, ymax = boxes[i]
        score = scores[i].item() if isinstance(scores, torch.Tensor) else scores
        if score > threshold:
            rect = patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none'
            )
            rects.append(rect)
    
    return img, rects

nrows = 8
ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))

batch_i = 0
for im, annot in test_data_loader:
    pos = batch_i * 4
    
    im_cpu = tuple(t.cpu() for t in im)
    boxes_cpu = annot[0]['boxes'].cpu()
    labels_cpu = annot[0]['labels'].cpu()

    preds_adj_all_cpu = []
    for preds_adj_dict in preds_adj_all[batch_i]:
        preds_adj_cpu = {
            'boxes': preds_adj_dict['boxes'].cpu(),
            'scores': preds_adj_dict['scores'].cpu()
        }
        preds_adj_all_cpu.append(preds_adj_cpu)

    for sample_i in range(len(im_cpu)):
        img_true, rects_true = plot_image_from_output_test(im_cpu[sample_i], boxes_cpu, 1.0)  # Use a placeholder score
        axes[pos // ncols, pos % ncols].imshow(img_true)

        for rect in rects_true:
            axes[pos // ncols, pos % ncols].add_patch(rect)
        
        if sample_i < len(preds_adj_all_cpu):
            img_pred, rects_pred = plot_image_from_output_test(im_cpu[sample_i], preds_adj_all_cpu[sample_i]['boxes'], preds_adj_all_cpu[sample_i]['scores'])
            axes[(pos + 1) // ncols, (pos + 1) % ncols].imshow(img_pred)

            for rect in rects_pred:
                axes[(pos + 1) // ncols, (pos + 1) % ncols].add_patch(rect)

        pos += 2

    batch_i += 1
    if batch_i == 4:
        break

# Remove xtick, ytick
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

colnames = ['True', 'Pred']
for idx, ax in enumerate(axes[0]):
    ax.set_title(colnames[idx])

plt.tight_layout()
plt.show()


In [ ]:
def plot_image_from_output_test(img, boxes, scores, threshold=0.5):
    img = img.cpu().permute(1, 2, 0).numpy()
    rects = []
    
    for i in range(boxes.size(0)):
        xmin, ymin, xmax, ymax = boxes[i]
        score = scores[i].item() if isinstance(scores, torch.Tensor) else scores
        if score > threshold:
            rect = patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none'
            )
            rects.append(rect)
    
    return img, rects

nrows = 8
ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))

batch_i = 0
for im, annot in test_data_loader:
    pos = batch_i * 2
    
    im_cpu = tuple(t.cpu() for t in im)
    boxes_cpu = annot[0]['boxes'].cpu()
    labels_cpu = annot[0]['labels'].cpu()

    preds_adj_all_cpu = []
    for preds_adj_dict in preds_adj_all[batch_i]:
        preds_adj_cpu = {
            'boxes': preds_adj_dict['boxes'].cpu(),
            'scores': preds_adj_dict['scores'].cpu()
        }
        preds_adj_all_cpu.append(preds_adj_cpu)

    for sample_i in range(len(im_cpu)):
        img_true, rects_true = plot_image_from_output_test(im_cpu[sample_i], boxes_cpu, 1.0)  # Use a placeholder score
        axes[pos // ncols, pos % ncols].imshow(img_true)

        for rect in rects_true:
            axes[pos // ncols, pos % ncols].add_patch(rect)
        
        if sample_i < len(preds_adj_all_cpu):
            img_pred, rects_pred = plot_image_from_output_test(im_cpu[sample_i], preds_adj_all_cpu[sample_i]['boxes'], preds_adj_all_cpu[sample_i]['scores'])
            axes[(pos + 1) // ncols, (pos + 1) % ncols].imshow(img_pred)

            for rect in rects_pred:
                axes[(pos + 1) // ncols, (pos + 1) % ncols].add_patch(rect)

        pos += 2

    batch_i += 1
    if batch_i == 4:
        break

# Remove xtick, ytick
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

colnames = ['True', 'Pred']
for idx, ax in enumerate(axes[0]):
    ax.set_title(colnames[idx])

plt.tight_layout()
plt.show()


In [ ]:
def plot_image_from_output_test(img, boxes, scores, threshold=0.5):
    img = img.cpu().permute(1, 2, 0).numpy()
    rects = []
    
    if isinstance(scores, torch.Tensor):
        scores = scores.cpu().numpy()
        for i in range(boxes.size(0)):
            xmin, ymin, xmax, ymax = boxes[i]
            score = scores[i].item() if len(scores) > i else 0.0
            if score > threshold:
                rect = patches.Rectangle(
                    (xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none'
                )
                rects.append(rect)
    else:
        if scores > threshold:
            xmin, ymin, xmax, ymax = boxes
            rect = patches.Rectangle(
                (xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='r', facecolor='none'
            )
            rects.append(rect)
    
    return img, rects


In [ ]:
nrows = 8
ncols = 2
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))

batch_i = 0
for im, annot in test_data_loader:
    pos = batch_i * 4
    
    im_cpu = tuple(t.cpu() for t in im)
    boxes_cpu = annot[0]['boxes'].cpu()
    labels_cpu = annot[0]['labels'].cpu()

    preds_adj_all_cpu = []
    for preds_adj_dict in preds_adj_all[batch_i]:
        preds_adj_cpu = {
            'boxes': preds_adj_dict['boxes'].cpu(),
            'scores': preds_adj_dict['scores'].cpu()
        }
        preds_adj_all_cpu.append(preds_adj_cpu)

    for sample_i in range(len(im_cpu)):
        true_boxes = boxes_cpu[sample_i].numpy()
        true_labels = labels_cpu[sample_i].numpy()
        
        img_true, rects_true = plot_image_from_output_test(im_cpu[sample_i], boxes_cpu, labels_cpu[sample_i])
        axes[pos // ncols, pos % ncols].imshow(img_true)

        for rect in rects_true:
            axes[pos // ncols, pos % ncols].add_patch(rect)
        
        if sample_i < len(preds_adj_all_cpu):
            pred_boxes = preds_adj_all_cpu[sample_i]['boxes'].numpy()
            pred_scores = preds_adj_all_cpu[sample_i]['scores'].numpy()
            
            img_pred, rects_pred = plot_image_from_output_test(im_cpu[sample_i], preds_adj_all_cpu[sample_i]['boxes'], preds_adj_all_cpu[sample_i]['scores'])
            axes[(pos + 1) // ncols, (pos + 1) % ncols].imshow(img_pred)

            for rect in rects_pred:
                axes[(pos + 1) // ncols, (pos + 1) % ncols].add_patch(rect)

            # Print image ID, true bounding boxes, and predicted boxes
            print(f"Image ID: {sample_i}")
            print("True Bounding Boxes:")
            for i, box in enumerate(true_boxes):
                print(f"  True Box {i + 1}: {box}, Label: {true_labels[i]}")
            print("Predicted Bounding Boxes:")
            for i, box in enumerate(pred_boxes):
                print(f"  Predicted Box {i + 1}: {box}, Score: {pred_scores[i]}")
            print("=" * 30)  # Separator between images
        
        pos += 2

    batch_i += 1
    if batch_i == 4:
        break

# Remove xtick, ytick
for idx, ax in enumerate(axes.flat):
    ax.set_xticks([])
    ax.set_yticks([])

colnames = ['True', 'Pred']
for idx, ax in enumerate(axes[0]):
    ax.set_title(colnames[idx])

plt.tight_layout()
plt.show()


In [ ]:
!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils /kaggle/working/Tutorial-Book-Utils

In [ ]:
!pwd

In [ ]:
!pip install gdown

In [ ]:
!python Tutorial-Book-Utils/PL_data_loader.py --data FaceMaskDetection

In [ ]:
!unzip -q Face\ Mask\ Detection.zip

In [ ]:
%cd /kaggle/working/Tutorial-Book-Utils
!ls

In [ ]:
import utils_ObjectDetection as utils

In [ ]:
%cd /kaggle/working/Tutorial-Book-Utils


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # all the batches get concatenated
precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
#mAP = torch.mean(AP)
#print(f'mAP : {mAP}')
print("Pred Scores : ", pred_scores)
print("Precision : ", precision)
mAP = torch.mean(AP.to(torch.float32))
print(f'mAP : {mAP.item()}')
print(f'AP : {AP}')

In [ ]:
preds_adj_all_cpu

In [ ]:
%cd /kaggle/working/Tutorial-Book-Utils
import utils_ObjectDetection as utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    preds_adj_all_cpu = [
        {
            'boxes': preds_adj_dict['boxes'].cpu(),
            'labels': preds_adj_dict['labels'].cpu(),
            'scores': preds_adj_dict['scores'].cpu()
        }
        for preds_adj_dict in preds_adj_all[batch_i]
    ]
    annot_all_cpu = [
        {
            'boxes': annot_dict['boxes'].cpu(),
            'labels': annot_dict['labels'].cpu()
        }
        for annot_dict in annot_all[batch_i]
    ]
    sample_metrics += utils.get_batch_statistics(preds_adj_all_cpu, annot_all_cpu, iou_threshold=0.5)

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]
precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels).cpu())
mAP = torch.mean(AP.to(torch.float32))
print(f'mAP : {mAP.item()}')
print(f'AP : {AP}')
